In [27]:
import yfinance as yf
import pandas as pd
import numpy as np

import os

import datetime as dt 
import utils 
import matplotlib.pyplot as plt

import pandas_ta as ta 

from backtesting import Backtest, Strategy

In [2]:
DATA_PATH = '../../data/stocks'

In [5]:
df = utils.download_and_cache(['CPSEETF.NS'], DATA_PATH)['CPSEETF.NS'].query('Volume > 0')
df

,Open,High,Low,Close,Volume
Date,,,,,
2014-04-04,18.000000,19.430000,18.000000,19.350000,83506926
2014-04-07,19.450001,19.730000,19.180000,19.360001,13205437
2014-04-09,19.400000,19.719999,19.350000,19.690001,4651893
2014-04-10,19.780001,19.879999,19.500000,19.820000,6410765
2014-04-11,19.820000,19.820000,19.500000,19.660000,1724842
...,...,...,...,...,...
2024-08-19,102.849998,103.190002,102.370003,102.830002,2799119
2024-08-20,103.400002,103.400002,102.400002,102.699997,2240676
2024-08-21,103.379997,103.379997,102.120003,102.739998,1724919


In [29]:
class SupertrendStrategy(Strategy):
    def supertrend(self, df, length):
        supertrend = ta.supertrend(df.High, df.Low, df.Close, length=length)
        return supertrend.to_numpy().T[1]

    def init(self):
        df = self.data.df
        self.dir = self.I(self.supertrend, df, 10)

    def next(self):
        if self.dir[-1] * self.dir[-2] > 0:
            return

        if self.dir[-1] > 0:
            self.buy()
        elif self.dir[-1] < 0:
            self.sell()

In [30]:
bt = Backtest(df, SupertrendStrategy, cash=1_00_000)
bt.run()

Start                     2014-04-04 00:00:00
End                       2024-08-23 00:00:00
Duration                   3794 days 00:00:00
Exposure Time [%]                   73.161189
Equity Final [$]                          0.0
Equity Peak [$]                 161759.096238
Return [%]                             -100.0
Buy & Hold Return [%]              429.043931
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]              117.974243
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                      -100.0
Avg. Drawdown [%]                  -14.358883
Max. Drawdown Duration     1612 days 00:00:00
Avg. Drawdown Duration      244 days 00:00:00
# Trades                                    6
Win Rate [%]                        33.333333
Best Trade [%]                      26.136359
Worst Trade [%]                   -101.068251
Avg. Trade [%]                    